In [1]:
# pip install numba --upgrade
# pip install pygraphviz
import sys
sys.path.insert(0, "/Users/yhhan/git/aiclass/0.Professor/")

import tensorflux.graph as tfg
import tensorflux.deep_learning_networks as tfn
import tensorflux.enums as tfe
import datasource.mnist as mnist
import tensorflux.functions as tff
import math
import numba
import numpy as np
import matplotlib.pyplot as plt
print(numba.__version__)
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflux'

In [ ]:
batch_size = 1000
max_epoch = 10
input_size = 784
hidden_layer1_size = 128
hidden_layer2_size = 128
output_size = 10

x = tfg.Placeholder(name="x")
target = tfg.Placeholder(name="target")

data = mnist.MNIST_Data(validation_size=5000, n_splits=12, is_onehot_target=True)
#data = mnist.Fashion_MNIST_Data(validation_size=5000, n_splits=12, is_onehot_target=True)

In [ ]:
n_xavier_normal = tfn.Multi_Layer_Network(
    input_size=input_size,
    hidden_size_list=[hidden_layer1_size, hidden_layer2_size],
    output_size=output_size,
    input_node=x,
    target_node=target,
    initializer=tfe.Initializer.Xavier_Normal.value,
    activator=tfe.Activator.ReLU.value,
    optimizer=tfe.Optimizer.Adam.value,
    learning_rate=0.001
)
n_xavier_normal.learning(
    max_epoch=max_epoch, 
    data=data, 
    batch_size=batch_size, 
    print_period=5, 
    is_numba=True, 
    verbose=False
)
data.reset_kfold()

In [ ]:
n_xavier_uniform = tfn.Multi_Layer_Network(
    input_size=input_size,
    hidden_size_list=[hidden_layer1_size, hidden_layer2_size],
    output_size=output_size,
    input_node=x,
    target_node=target,
    initializer=tfe.Initializer.Xavier_Uniform.value,
    activator=tfe.Activator.ReLU.value,
    optimizer=tfe.Optimizer.Adam.value,
    learning_rate=0.001
)
n_xavier_uniform.learning(
    max_epoch=max_epoch, 
    data=data, 
    batch_size=batch_size, 
    print_period=5, 
    is_numba=True, 
    verbose=False
)
data.reset_kfold()

In [ ]:
n_he_normal = tfn.Multi_Layer_Network(
    input_size=input_size,
    hidden_size_list=[hidden_layer1_size, hidden_layer2_size],
    output_size=output_size,
    input_node=x,
    target_node=target,
    initializer=tfe.Initializer.He_Normal.value,
    activator=tfe.Activator.ReLU.value,
    optimizer=tfe.Optimizer.Adam.value,
    learning_rate=0.001
)
n_he_normal.learning(
    max_epoch=max_epoch, 
    data=data, 
    batch_size=batch_size, 
    print_period=5, 
    is_numba=True, 
    verbose=False
)
data.reset_kfold()

In [ ]:
n_he_uniform = tfn.Multi_Layer_Network(
    input_size=input_size,
    hidden_size_list=[hidden_layer1_size, hidden_layer2_size],
    output_size=output_size,
    input_node=x,
    target_node=target,
    initializer=tfe.Initializer.He_Uniform.value,
    activator=tfe.Activator.ReLU.value,
    optimizer=tfe.Optimizer.Adam.value,
    learning_rate=0.001
)
n_he_uniform.learning(
    max_epoch=max_epoch, 
    data=data, 
    batch_size=batch_size, 
    print_period=5, 
    is_numba=True, 
    verbose=False
)
data.reset_kfold()

In [ ]:
neural_networks = {
    "xavier_normal": n_xavier_normal, 
    "xavier_uniform": n_xavier_uniform,     
    "he_normal": n_he_normal,         
    "he_uniform": n_he_uniform
}

In [ ]:
for key, neural_network in neural_networks.items():
    print("{:10s} - Epoch:{:3d}, Min Train Error:{:7.5f}, Min Validation Error:{:7.5f}, Test Accuracy:{:7.5f}".format(
        key, 
        neural_network.min_validation_error_epoch,
        neural_network.min_train_error,
        neural_network.min_validation_error,
        neural_network.test_accuracy_at_min_validation_error_epoch
    ))

print()
    
for key, neural_network in neural_networks.items():    
    print("{:10s} - Mean of Min Va. Error For All Folds:{:7.5f}".format(
        key,         
        neural_network.mean_min_validation_error_for_all_folds
    ))

In [ ]:
markers = {
    "xavier_normal": "+", 
    "xavier_uniform": "d",     
    "he_normal": "*",         
    "he_uniform": "o"
}

epoch_list = np.arange(len(neural_networks["xavier_normal"].train_error_list))

f, axarr = plt.subplots(2, 2, figsize=(15,10))

for key, neural_network in neural_networks.items():
    axarr[0, 0].plot(epoch_list, neural_network.train_error_list, marker=markers[key], markevery=2, label=key)
axarr[0, 0].set_ylabel('Train Error')
axarr[0, 0].set_xlabel('Epochs')
axarr[0, 0].grid(True)
axarr[0, 0].set_title('Train Error')
axarr[0, 0].legend(loc='upper left')

for key, neural_network in neural_networks.items():
    axarr[0, 1].plot(epoch_list, neural_network.validation_error_list, marker=markers[key], markevery=2, label=key)
axarr[0, 1].set_ylabel('Validation Error')
axarr[0, 1].set_xlabel('Epochs')
axarr[0, 1].grid(True)
axarr[0, 1].set_title('Validation Error')
axarr[0, 1].legend(loc='upper left')

for key, neural_network in neural_networks.items():
    axarr[1, 0].plot(epoch_list, neural_network.train_error_list, marker=markers[key], markevery=2, label=key)
axarr[1, 0].set_ylabel('Train Error')
axarr[1, 0].set_xlabel('Epochs')
axarr[1, 0].grid(True)
axarr[1, 0].set_ylim(0, 0.15)
axarr[1, 0].set_title('Train Error (0.00 ~ 0.15)')
axarr[1, 0].legend(loc='upper left')

for key, neural_network in neural_networks.items():
    axarr[1, 1].plot(epoch_list, neural_network.validation_error_list, marker=markers[key], markevery=2, label=key)
axarr[1, 1].set_ylabel('Validation Error')
axarr[1, 1].set_xlabel('Epochs')
axarr[1, 1].grid(True)
axarr[1, 1].set_ylim(0, 0.15)
axarr[1, 1].set_title('Validation Error (0.00 ~ 0.15)')
axarr[1, 1].legend(loc='upper left')

f.subplots_adjust(hspace=0.3)

plt.show()

In [ ]:
f, axarr = plt.subplots(2, 1, figsize=(15,10))

for key, neural_network in neural_networks.items():
    axarr[0].plot(epoch_list, neural_network.test_accuracy_list, marker=markers[key], markevery=1, label=key)
axarr[0].set_ylabel('Test Accuracy')
axarr[0].set_xlabel('Epochs')
axarr[0].grid(True)
axarr[0].set_title('Test Accuracy')
axarr[0].legend(loc='upper left')

for key, neural_network in neural_networks.items():
    axarr[1].plot(epoch_list, neural_network.test_accuracy_list, marker=markers[key], markevery=1, label=key)
axarr[1].set_ylabel('Test Accuracy')
axarr[1].set_xlabel('Epochs')
axarr[1].grid(True)
axarr[1].set_ylim(0.95, 0.99)
axarr[1].set_title('Test Accuracy (0.95 ~ 0.99)')
axarr[1].legend(loc='upper left')

f.subplots_adjust(hspace=0.3)
plt.show()

In [ ]:
neural_networks["xavier_normal"].draw_param_description(figsize=(20, 10))

In [ ]:
neural_networks["xavier_normal"].draw_output_description(figsize=(20, 10))